In [80]:
# Libraries setup
import os
import warnings
import optuna

import pandas as pd
import numpy as np

from dotenv import load_dotenv
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

warnings.filterwarnings('ignore')
load_dotenv()

True

### Data Load Pipeline
The data load pipeline is almost the same as in EDA. For the purpose of demonstration, it is not a new module, but placed as a function.

In [81]:
# Define paths
fpath_data = os.environ.get("FPATH_DATA")
fpath_dicts = os.environ.get("FPATH_DICTS")

def get_data_order_items():
    # Load order items data
    order_items = pd.read_csv(fpath_data+"order_items.csv")
    # Convert shipping_limit_date to timestamps
    order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])
    # Lets load data regarding product category, drop redundant columns, check the data, and then join the data to order_items
    product_categories = pd.read_csv(fpath_data+"products.csv")
    product_categories = product_categories[["product_id", "product_category_name"]]
    # Drop nulls
    product_categories = product_categories.dropna()
    # Map categories to english ones
    categories_translation_data = pd.read_csv(fpath_data+"product_category_name_translation.csv")
    product_categories = pd.merge(product_categories, categories_translation_data, how='left', on='product_category_name')
    # Instead of dropping 13 rows w/o translation, use the same data as originally
    product_categories['product_category_name_english'] = product_categories['product_category_name_english'].fillna(product_categories['product_category_name'])
    # Join product category to orders
    order_items = pd.merge(order_items, product_categories, how='left', on='product_id')
    # I decided to drop those without any category prescribed as those might be incorrect data
    order_items = order_items.dropna(subset=['product_category_name'])
    # Join data regarding purchase timestamp - first, read data
    orders = pd.read_csv(fpath_data+'orders.csv')
    # Keep only relevant columns
    orders = orders[['order_id','order_purchase_timestamp']]
    # Check for nulls
    orders.isnull().sum()
    # Transform order_purchase_timestamp to a timestamp object
    orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
    # Join orders data via inner - we could do also left join, but it would need to double check for nulls
    order_items = pd.merge(order_items, orders, how='inner', on='order_id')
    # Drop redundant data, not related to this task
    # As this is just a demonstration, we won't be adding information such as holidays for a different set of countries the customers are from etc
    order_items = order_items.drop(columns=['freight_value','shipping_limit_date','product_category_name'])
    # Add a set of information regarding sale timestamp
    order_items['Purchase Date'] = pd.to_datetime(order_items['order_purchase_timestamp']).dt.floor('D')
    return order_items

In [82]:
order_items = get_data_order_items()

dataset = order_items.groupby(['product_category_name_english', 'Purchase Date']).agg(
    sold_products_quantity=('order_item_id', 'size')
    # sellers_count=('seller_id', 'count')
).sort_values(by=['product_category_name_english', 'Purchase Date'], ascending=True).reset_index()
dataset

,product_category_name_english,Purchase Date,sold_products_quantity
0,agro_industry_and_commerce,2017-01-23,2
1,agro_industry_and_commerce,2017-01-31,1
2,agro_industry_and_commerce,2017-02-05,1
3,agro_industry_and_commerce,2017-02-08,1
4,agro_industry_and_commerce,2017-02-12,1
...,...,...,...
18499,watches_gifts,2018-08-25,3
18500,watches_gifts,2018-08-26,2
18501,watches_gifts,2018-08-27,2
18502,watches_gifts,2018-08-28,1


### Features Engineering
Feature engineering is the process of creating new features or transforming existing ones to improve 
the performance of machine learning models. 

##### Definition of new features functions

In [83]:
# Time Based Features
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()
        X_['dayofweek'] = X_.index.dayofweek
        X_['quarter'] = X_.index.quarter
        X_['month'] = X_.index.month
        X_['year'] = X_.index.year
        X_['dayofyear'] = X_.index.dayofyear
        X_['dayofmonth'] = X_.index.day
        X_['weekofyear'] = X_.index.isocalendar().week
        X_['product_category_name_english'] = X_['product_category_name_english'].astype('category')
        return X_

# # Lag Features
# def create_lag_features(df, column, lags):
#     for lag in lags:
#         df[f'{column}_lag_{lag}'] = df.groupby('product_category_name_english')[column].shift(lag)
#     return df

# # Rolling Statistics
# def create_rolling_features(df, column, windows):
#     for window in windows:
#         df[f'{column}_rolling_mean_{window}'] = df.groupby('product_category_name_english')[column].rolling(window=window).mean().reset_index(0,drop=True)
#         df[f'{column}_rolling_std_{window}'] = df.groupby('product_category_name_english')[column].rolling(window=window).std().reset_index(0,drop=True)
#     return df

# # Exponential Moving Average
# def create_ema_features(df, column, spans):
#     for span in spans:
#         df[f'{column}_ema_{span}'] = df.groupby('product_category_name_english')[column].ewm(span=span).mean().reset_index(0,drop=True)
#     return df

# # Fourier Terms for Seasonality
# def fourier_series(dates, period, order):
#     t = (dates - pd.Timestamp("1970-01-01")) / pd.Timedelta('1D')
#     return pd.DataFrame({f'fourier_cos_{period}_{n}': np.cos(2 * n * np.pi * t / period),
#                          f'fourier_sin_{period}_{n}': np.sin(2 * n * np.pi * t / period)}
#                         for n in range(1, order + 1))

# # Price-related Features
# def price_related_features(df):
#     df['price_rolling_mean'] = df.groupby('product_category_name_english')['price'].rolling(window=7).mean().reset_index(0,drop=True)
#     df['price_relative_to_mean'] = df['price'] / df['price_rolling_mean']
#     return df

# # Sold Items Quantity Features
# def quantity_related_features(df):
#     # Sort the dataframe by timestamp and category
#     df = df.sort_values(['product_category_name_english', 'order_purchase_timestamp'])
#     # Calculate rolling count of sold items quantity
#     df['quantity_rolling_count'] = df.groupby('product_category_name_english')['order_item_id'].transform(
#         lambda x: x.rolling(window='7D').count()
#     )
#     # Calculate daily quantity
#     df['daily_quantity'] = df.groupby(['product_category_name_english', df['order_purchase_timestamp'].dt.date])['order_item_id'].transform('count')
#     # Calculate quantity relative to rolling count
#     df['quantity_relative_to_count'] = df['daily_quantity'] / df['quantity_rolling_count']
#     # Calculate cumulative sum of sold items within each category
#     df['cumulative_quantity'] = df.groupby('product_category_name_english')['daily_quantity'].cumsum()
#     return df

##### Final data pipeline that leverages functions written above

In [84]:
def prepare_data(dataset):
    dataset['Purchase Date'] = pd.to_datetime(dataset['Purchase Date'])
    daily_sales = dataset.groupby(['Purchase Date', 'product_category_name_english'])['sold_products_quantity'].sum().reset_index()
    return daily_sales.sort_values(['product_category_name_english', 'Purchase Date'])

def create_forecast_data(last_date, forecast_end, category):
    forecast_start = last_date + pd.Timedelta(days=1)
    forecast_dates = pd.date_range(start=forecast_start, end=forecast_end, freq='D')
    forecast_data = pd.DataFrame({'Purchase Date': forecast_dates, 'product_category_name_english': category})
    forecast_data.set_index('Purchase Date', inplace=True)
    return forecast_data

##### Evaluation metrics

In [85]:
def wmae(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred) * np.arange(1, len(y_true)+1)) / np.sum(np.arange(1, len(y_true)+1))

def bias(y_true, y_pred):
    return np.mean(y_pred - y_true)

##### Optuna - automated xgb hyperparameter search

In [86]:
def objective(trial, X_train, y_train, X_val, y_val):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'enable_categorical': True
    }
    
    model = XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
    
    preds = model.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    return mae

##### Main function

In [87]:
def xgboost_forecast(data, category, forecast_end='2018-09-24'):
    data = data.set_index('Purchase Date')
    
    features = ['product_category_name_english', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear']
    
    fe = FeatureEngineer()
    data_transformed = fe.fit_transform(data)
    
    split_date = data.index.max() - pd.Timedelta(days=30)
    train_data = data_transformed[data_transformed.index <= split_date]
    val_data = data_transformed[data_transformed.index > split_date]
    
    X_train, y_train = train_data[features], train_data['sold_products_quantity']
    X_val, y_val = val_data[features], val_data['sold_products_quantity']
    
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=50)
    
    best_params = study.best_params
    best_params['enable_categorical'] = True
    
    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train)
    
    in_sample_forecast = model.predict(X_val)
    
    last_date = data.index.max()
    forecast_data = create_forecast_data(last_date, forecast_end, category)
    forecast_data_transformed = fe.transform(forecast_data)
    out_of_sample_forecast = model.predict(forecast_data_transformed[features])
    forecast_data['forecast'] = out_of_sample_forecast
    
    return val_data, in_sample_forecast, forecast_data, best_params

##### Main execution

In [88]:
data = prepare_data(dataset)

forecasts = {}
metrics = {}
best_params = {}

for category in tqdm(data['product_category_name_english'].unique(), desc="Processing categories"):
    category_data = data[data['product_category_name_english'] == category]
    
    last_date = category_data['Purchase Date'].max()
    print(f"\nCategory: {category}, Last date: {last_date}")
    
    if last_date >= pd.Timestamp('2018-09-24'):
        print(f"Skipping forecast for {category} as data is available up to or beyond 2018-09-24")
        continue
    
    test_data, in_sample_forecast, forecast_data, params = xgboost_forecast(category_data, category, forecast_end='2018-09-24')
    
    mae = mean_absolute_error(test_data['sold_products_quantity'], in_sample_forecast)
    wmae_score = wmae(test_data['sold_products_quantity'].values, in_sample_forecast)
    bias_score = bias(test_data['sold_products_quantity'].values, in_sample_forecast)
    
    metrics[category] = {
        'MAE': mae,
        'WMAE': wmae_score,
        'Bias': bias_score,
        'Last_Date': last_date
    }
    
    forecasts[category] = forecast_data['forecast']
    best_params[category] = params

Processing categories:   0%|          | 0/73 [00:00<?, ?it/s][I 2024-06-28 01:19:10,650] A new study created in memory with name: no-name-eb1fad5d-108e-4efd-9980-55d893eb8422


[I 2024-06-28 01:19:10,797] Trial 0 finished with value: 0.7861618144171578 and parameters: {'max_depth': 8, 'learning_rate': 0.0035838286986023767, 'n_estimators': 610, 'min_child_weight': 4, 'subsample': 0.5855094682787634, 'colsample_bytree': 0.7622059326505335}. Best is trial 0 with value: 0.7861618144171578.



Category: agro_industry_and_commerce, Last date: 2018-08-26 00:00:00


[I 2024-06-28 01:19:10,923] Trial 1 finished with value: 0.8593347242900303 and parameters: {'max_depth': 9, 'learning_rate': 0.2904026744051031, 'n_estimators': 498, 'min_child_weight': 2, 'subsample': 0.7378908840308921, 'colsample_bytree': 0.9642094252108573}. Best is trial 0 with value: 0.7861618144171578.
[I 2024-06-28 01:19:10,988] Trial 2 finished with value: 0.8136925952775138 and parameters: {'max_depth': 3, 'learning_rate': 0.026271478814375273, 'n_estimators': 397, 'min_child_weight': 2, 'subsample': 0.8846295699132279, 'colsample_bytree': 0.541670774548757}. Best is trial 0 with value: 0.7861618144171578.
[I 2024-06-28 01:19:11,054] Trial 3 finished with value: 0.783311128616333 and parameters: {'max_depth': 2, 'learning_rate': 0.0024594949034280056, 'n_estimators': 314, 'min_child_weight': 1, 'subsample': 0.9412833483833227, 'colsample_bytree': 0.8958081368348632}. Best is trial 3 with value: 0.783311128616333.
[I 2024-06-28 01:19:11,138] Trial 4 finished with value: 0.803


Category: air_conditioning, Last date: 2018-08-25 00:00:00


...

[I 2024-06-28 01:26:54,445] Trial 0 finished with value: 5.011527061462402 and parameters: {'max_depth': 1, 'learning_rate': 0.004903879711298548, 'n_estimators': 511, 'min_child_weight': 4, 'subsample': 0.640958760283829, 'colsample_bytree': 0.5766469311398388}. Best is trial 0 with value: 5.011527061462402.
[I 2024-06-28 01:26:54,546] Trial 1 finished with value: 5.324601825078329 and parameters: {'max_depth': 4, 'learning_rate': 0.003784038496279552, 'n_estimators': 314, 'min_child_weight': 4, 'subsample': 0.9277181812690068, 'colsample_bytree': 0.8399542014811949}. Best is trial 0 with value: 5.011527061462402.
[I 2024-06-28 01:26:54,662] Trial 2 finished with value: 5.327489487330118 and parameters: {'max_depth': 8, 'learning_rate': 0.003414015192276234, 'n_estimators': 172, 'min_child_weight': 7, 'subsample': 0.6774228373525015, 'colsample_bytree': 0.8470784546672845}. Best is trial 0 with value: 5.011527061462402.
[I 2024-06-28 01:26:54,726] Trial 3 finished with value: 5.508830


Category: toys, Last date: 2018-08-29 00:00:00


[I 2024-06-28 01:27:02,264] Trial 1 finished with value: 2.6093638965061734 and parameters: {'max_depth': 9, 'learning_rate': 0.022778394957592336, 'n_estimators': 189, 'min_child_weight': 4, 'subsample': 0.8659852336194604, 'colsample_bytree': 0.7447135730102092}. Best is trial 0 with value: 2.496310523578099.
[I 2024-06-28 01:27:02,401] Trial 2 finished with value: 2.6525235857282365 and parameters: {'max_depth': 1, 'learning_rate': 0.4170431120024398, 'n_estimators': 292, 'min_child_weight': 9, 'subsample': 0.7149820079018802, 'colsample_bytree': 0.8492513492866121}. Best is trial 0 with value: 2.496310523578099.
[I 2024-06-28 01:27:02,509] Trial 3 finished with value: 2.63645339012146 and parameters: {'max_depth': 9, 'learning_rate': 0.08586536271518384, 'n_estimators': 481, 'min_child_weight': 3, 'subsample': 0.54777076333771, 'colsample_bytree': 0.7920898788398709}. Best is trial 0 with value: 2.496310523578099.
[I 2024-06-28 01:27:02,608] Trial 4 finished with value: 2.429930942


Category: watches_gifts, Last date: 2018-08-29 00:00:00


[I 2024-06-28 01:27:09,393] Trial 1 finished with value: 7.392829545338949 and parameters: {'max_depth': 7, 'learning_rate': 0.487973542550557, 'n_estimators': 804, 'min_child_weight': 4, 'subsample': 0.7845754977389112, 'colsample_bytree': 0.6126748709607273}. Best is trial 0 with value: 6.850170167287191.
[I 2024-06-28 01:27:09,708] Trial 2 finished with value: 7.098444588979086 and parameters: {'max_depth': 2, 'learning_rate': 0.002060850878737972, 'n_estimators': 714, 'min_child_weight': 8, 'subsample': 0.6321988238630055, 'colsample_bytree': 0.9765844982214463}. Best is trial 0 with value: 6.850170167287191.
[I 2024-06-28 01:27:10,150] Trial 3 finished with value: 7.42089262008667 and parameters: {'max_depth': 6, 'learning_rate': 0.00120351866940593, 'n_estimators': 647, 'min_child_weight': 8, 'subsample': 0.5946106732439361, 'colsample_bytree': 0.6794972730497069}. Best is trial 0 with value: 6.850170167287191.
[I 2024-06-28 01:27:10,278] Trial 4 finished with value: 6.9535695393

In [89]:
# Create DataFrames with all forecasts and metrics
forecast_df = pd.DataFrame(forecasts)
metrics_df = pd.DataFrame(metrics).T
params_df = pd.DataFrame(best_params).T

print("\nForecasts up to 2018-09-24:")
print(forecast_df)
print("\nMetrics for each category:")
print(metrics_df)
print("\nBest parameters for each category:")
print(params_df)

forecast_df.to_csv('XGBoost_category_demand_forecasts.csv')
metrics_df.to_csv('XGBoost_category_demand_metrics.csv')
params_df.to_csv('XGBoost_category_demand_params.csv')
print("Forecasts have been saved to 'XGBoost_category_demand_forecasts.csv'")
print("Metrics have been saved to 'XGBoost_category_demand_metrics.csv'")
print("Params have been saved to 'XGBoost_category_demand_params.csv'")


Forecasts up to 2018-09-24:
               agro_industry_and_commerce  air_conditioning       art  \
Purchase Date                                                           
2017-09-18                            NaN               NaN       NaN   
2017-09-19                            NaN               NaN       NaN   
2017-09-20                            NaN               NaN       NaN   
2017-09-21                            NaN               NaN       NaN   
2017-09-22                            NaN               NaN       NaN   
...                                   ...               ...       ...   
2018-09-20                       1.522938          1.128028  0.442861   
2018-09-21                       1.522938          1.333061 -0.899955   
2018-09-22                       1.522938          1.079084 -1.135709   
2018-09-23                       1.522938          1.505667 -0.072002   
2018-09-24                       1.522938          0.984530 -0.604590   

               arts_a